In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import codecs as cd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, confusion_matrix
import seaborn as sns

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split, GridSearchCV
from IPython.display import display
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,classification_report
from tensorflow import keras  
import tensorflow as tf

from datetime import datetime
from datetime import date

In [2]:
#201801のデータを読み込んでみる
df = pd.read_csv('ks-projects-201801.csv')
#最初の五行を表示してみる。
display(df.head())

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [3]:
#開催日数を追加
df["launched"] = df["launched"].apply(lambda x:datetime.strptime(x[:10], "%Y-%m-%d"))
df["deadline"] = df["deadline"].apply(lambda x:datetime.strptime(x[:10], "%Y-%m-%d"))

day_difference = (df["deadline"]-df["launched"]).apply(lambda x: x.days)
df['day']=day_difference

In [4]:
#統計値を表示してみる。
display(df.describe())
#欠損値がある行数を数えてみる。
display(df.isnull().sum())
#各列のデータタイプを確認してみる。
display(df.dtypes)
#最初の五行を表示してみる。
display(df.head())

,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real,day
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05,378661.000000
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04,34.481095
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06,65.909173
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02,1.000000
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03,30.000000
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03,30.000000
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04,37.000000
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08,16739.000000


ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
day                    0
dtype: int64

ID                           int64
name                        object
category                    object
main_category               object
currency                    object
deadline            datetime64[ns]
goal                       float64
launched            datetime64[ns]
pledged                    float64
state                       object
backers                      int64
country                     object
usd pledged                float64
usd_pledged_real           float64
usd_goal_real              float64
day                          int64
dtype: object

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,day
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11,0.0,failed,0,GB,0.0,0.0,1533.95,59
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02,2421.0,failed,15,US,100.0,2421.0,30000.00,60
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12,220.0,failed,3,US,220.0,220.0,45000.00,45
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17,1.0,failed,1,US,1.0,1.0,5000.00,30
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04,1283.0,canceled,14,US,1283.0,1283.0,19500.00,56


In [5]:
#ダミー変数中身
X_cols=['category','main_category','currency','country']
y_col='state'
#ダミー変数作成
X=pd.get_dummies(df[X_cols])
y=pd.get_dummies(df[y_col])['successful']

X['day']=df['day']
X

,category_3D Printing,category_Academic,category_Accessories,category_Action,category_Animals,category_Animation,category_Anthologies,category_Apparel,category_Apps,category_Architecture,...,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,day
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,59
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,60
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,45
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,30
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,56
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,35
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,20
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,45
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,35
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,30


In [6]:
# estimatorにモデルをセット
estimator = SGDClassifier(loss="log")

# RFECVは交差検証によってステップワイズ法による特徴選択を行う
# cvにはFold（=グループ）の数，scoringには評価指標を指定する
#今回は回帰なのでneg_mean_absolute_errorを評価指標に指定（分類ならaccuracy）cv=クロスバリデーション
rfecv = RFECV(estimator, cv=10, scoring='accuracy')#再帰的な機能の削除と、最適な数の機能の相互検証された選択による機能のランキング。

In [7]:
#特徴選択
rfecv.fit(X, y)

RFECV(cv=10,
      estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                              early_stopping=False, epsilon=0.1, eta0=0.0,
                              fit_intercept=True, l1_ratio=0.15,
                              learning_rate='optimal', loss='log',
                              max_iter=1000, n_iter_no_change=5, n_jobs=None,
                              penalty='l2', power_t=0.5, random_state=None,
                              shuffle=True, tol=0.001, validation_fraction=0.1,
                              verbose=0, warm_start=False),
      min_features_to_select=1, n_jobs=None, scoring='accuracy', step=1,
      verbose=0)

In [8]:
# bool型の配列に ~ をつけるとTrueとFalseを反転させることができる
# ここでTrueになっている特徴が削除してもよい特徴
remove_idx = ~rfecv.support_
remove_idx

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False,  True,
       False,  True,  True, False, False, False, False, False, False,
       False,  True, False, False, False,  True, False, False, False,
       False, False, False, False,  True, False, False,  True, False,
       False,  True, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [9]:
# 削除してもよい特徴の名前を取得する
remove_feature = X.columns[remove_idx]
remove_feature

Index(['category_Blues', 'category_Children's Books', 'category_Comics',
       'category_Conceptual Art', 'category_Cookbooks', 'category_Design',
       'category_Drinks', 'category_Fantasy', 'category_Fashion',
       'category_Film & Video', 'category_Glass', 'category_Jewelry',
       'category_Kids', 'category_Music Videos', 'category_Pet Fashion',
       'category_Poetry', 'category_Sculpture', 'category_World Music',
       'main_category_Art', 'main_category_Publishing', 'currency_HKD',
       'currency_SEK', 'country_AU', 'country_DE', 'country_GB', 'country_HK',
       'country_IE', 'country_SE', 'day'],
      dtype='object')

In [10]:
# drop関数で特徴を削除
Sort_X= X.drop(remove_feature, axis=1)

In [11]:
#ロジスティック回帰で学習
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234)
clf.fit(Sort_X, y)

# 学習した結果を使って説明変数を入力して予測
y_est = clf.predict(Sort_X)

#混同行列を作成
tn, fp, fn, tp = confusion_matrix(y, y_est).ravel()
print(fn, fp)
print(tn, tp)

#'Accuracy、Recall、Precisionを求めて表示
print('Accuracy  = {:.3f}%'.format(100 * (tn+tp)/(tn+fp+fn+tp)))
print('Recall    = {:.3f}%'.format(100 * (tp)/(fn+tp)))
print('Precision = {:.3f}%'.format(100 * (tp)/(fp+tp)))

93050 29605
215100 40906
Accuracy  = 67.608%
Recall    = 30.537%
Precision = 58.014%


In [12]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
clf = DecisionTreeClassifier(criterion="gini", max_depth=None, min_samples_split=3, min_samples_leaf=3, random_state=1234)
clf = clf.fit(Sort_X, y)
# print("score=", clf.score(X, y))
# print(clf.predict(X_test)) #予測したい場合

# 学習した結果を使って説明変数を入力して予測
y_est = clf.predict(Sort_X)

#混同行列を作成
tn, fp, fn, tp = confusion_matrix(y, y_est).ravel()
print(fn, fp)
print(tn, tp)

#'Accuracy、Recall、Precisionを求めて表示
print('Accuracy  = {:.3f}%'.format(100 * (tn+tp)/(tn+fp+fn+tp)))
print('Recall    = {:.3f}%'.format(100 * (tp)/(fn+tp)))
print('Precision = {:.3f}%'.format(100 * (tp)/(fp+tp)))

88463 33275
211430 45493
Accuracy  = 67.850%
Recall    = 33.961%
Precision = 57.756%


In [13]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, max_depth=2, criterion="gini",
                                                 min_samples_leaf=2, min_samples_split=2, random_state=1234)
clf.fit(Sort_X, y)

# 学習した結果を使って説明変数を入力して予測
y_est = clf.predict(Sort_X)

#混同行列を作成
tn, fp, fn, tp = confusion_matrix(y, y_est).ravel()
print(fn, fp)
print(tn, tp)

#'Accuracy、Recall、Precisionを求めて表示
print('Accuracy  = {:.3f}%'.format(100 * (tn+tp)/(tn+fp+fn+tp)))
print('Recall    = {:.3f}%'.format(100 * (tp)/(fn+tp)))
print('Precision = {:.3f}%'.format(100 * (tp)/(fp+tp)))

133956 0
244705 0
Accuracy  = 64.624%
Recall    = 0.000%
Precision = nan%


C:\Users\N395\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in longlong_scalars


In [14]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3,
                                                min_samples_leaf=2,
                                                min_samples_split=2, 
                                                random_state=1234,
                                                criterion="gini"),
                                                n_estimators=10,
                                                random_state=1234)
clf.fit(Sort_X, y)

# 学習した結果を使って説明変数を入力して予測
y_est = clf.predict(Sort_X)

#混同行列を作成
tn, fp, fn, tp = confusion_matrix(y, y_est).ravel()
print(fn, fp)
print(tn, tp)

#'Accuracy、Recall、Precisionを求めて表示
print('Accuracy  = {:.3f}%'.format(100 * (tn+tp)/(tn+fp+fn+tp)))
print('Recall    = {:.3f}%'.format(100 * (tp)/(fn+tp)))
print('Precision = {:.3f}%'.format(100 * (tp)/(fp+tp)))

93198 30383
214322 40758
Accuracy  = 67.364%
Recall    = 30.426%
Precision = 57.292%
